In [1]:
from pathlib import Path
from src.data.data_loader import Species, load_dataset, PyTorchDataset
from src.features.encoder import FeatureEncoder
from src.utils.torch2sklearn import Torch2SkLearn

from src.models.reporting.model_report import ModelReport
from src.models.reporting.single_report import generate_report

In [2]:
x, y = load_dataset(Species.human, encoding=FeatureEncoder.binary)
x_test, y_test = load_dataset(Species.human, independent=True, encoding=FeatureEncoder.binary)

In [3]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
regressor = LogisticRegressionCV()
regressor.fit(x, y)

generate_report(
    {'train': ModelReport.generate(regressor, x, y), 'test': ModelReport.generate(regressor, x_test, y_test)},
    Path('lr_simple'))

In [ ]:
imp_x = x[map(lambda _: f'binary_{_}', [46, 36, 44, 77, 45, 49, 3])]
imp_x_test = x_test[map(lambda _: f'binary_{_}', [46, 36, 44, 77, 45, 49, 3])]
regressor2 = LogisticRegressionCV()
regressor2.fit(imp_x, y)

generate_report(
    {'train': ModelReport.generate(regressor2, imp_x, y), 'test': ModelReport.generate(regressor2, imp_x_test, y_test)},
    Path('lr_feature_selection'))

# Tensorflow Portion

In [45]:
from sklearn.model_selection import train_test_split

from keras.layers import Dense
from keras.models import Sequential
from keras.src.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(X_train.shape)
# Define the TensorFlow model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

(792, 84)


In [46]:
model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
25/25 [==============================] - 0s 675us/step - loss: 0.7097 - accuracy: 0.5126
Epoch 2/100
25/25 [==============================] - 0s 614us/step - loss: 0.6831 - accuracy: 0.5530
Epoch 3/100
25/25 [==============================] - 0s 613us/step - loss: 0.6640 - accuracy: 0.5934
Epoch 4/100
25/25 [==============================] - 0s 590us/step - loss: 0.6490 - accuracy: 0.6364
Epoch 5/100
25/25 [==============================] - 0s 539us/step - loss: 0.6361 - accuracy: 0.6465
Epoch 6/100
25/25 [==============================] - 0s 539us/step - loss: 0.6239 - accuracy: 0.6818
Epoch 7/100
25/25 [==============================] - 0s 525us/step - loss: 0.6135 - accuracy: 0.6843
Epoch 8/100
25/25 [==============================] - 0s 475us/step - loss: 0.6047 - accuracy: 0.6818
Epoch 9/100
25/25 [==============================] - 0s 562us/step - loss: 0.5946 - accuracy: 0.7033
Epoch 10/100
25/25 [==============================] - 0s 493us/step - loss: 0.5858 - accura

In [63]:
y_scores = model.predict(X_test)
y_pred = (y_scores > 0.5).astype(int)

print(classification_report(y_train, model.predict_classes(X_train)))
# print(classification_report(y_test, model.predict(X_test)))

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [61]:
model.predict([X_train.iloc[0]])

ValueError: in user code:

    File "/Users/arish/workspace/research/psi_predictor/venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/Users/arish/workspace/research/psi_predictor/venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/arish/workspace/research/psi_predictor/venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/arish/workspace/research/psi_predictor/venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/Users/arish/workspace/research/psi_predictor/venv/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.w

In [59]:
X_train

,binary_1,binary_2,binary_3,binary_4,binary_5,binary_6,binary_7,binary_8,binary_9,binary_10,...,binary_75,binary_76,binary_77,binary_78,binary_79,binary_80,binary_81,binary_82,binary_83,binary_84
753,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
215,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
294,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
49,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
720,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
270,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
860,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
435,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
